In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import pandas as pd

In [ ]:
older_path = '/content/drive/MyDrive/Chatgpt_KMLE/input'

file_list = os.listdir(folder_path)

case_list = []
acc_list = []

csv_files = [file for file in file_list if file.endswith('.csv')]
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    try:
        df = pd.read_csv(file_path)
    except:
        df = pd.read_csv(file_path, encoding="utf-8-sig")

    df["GPT_answer"] = df["GPT_answer"].apply(lambda x : str(x).replace("[", "").replace("]", "").replace(" ",""))
    df["score"] = 1
    df2 = pd.pivot_table(data=df,index=["questionnum", "GPT_answer"], values=["score"], aggfunc="count").reset_index()
    df3 = df2.loc[df2.groupby('questionnum')['score'].idxmax()]
    df4 = pd.merge(df3, df[["questionnum", "Answer"]].drop_duplicates(), how="left", on="questionnum")[["questionnum", "Answer", "GPT_answer", "score"]]
    df4.columns = ['questionnum', 'Answer', 'GPT_answer', 'Count']
    df4["check"] = df4.apply(lambda x : 1 if x["Answer"] == x["GPT_answer"] else 0, 1)
    merge1 = df[["questionnum", "GPT_answer", "GPT_reasoning"]].sort_values(["questionnum", "GPT_reasoning"], ascending=[True, False]).groupby(["questionnum", "GPT_answer"]).first().reset_index()
    df5 = pd.merge(df4, merge1, how="left", on=["questionnum", "GPT_answer"]).sort_values("questionnum")

    df5.to_excel(f"/content/drive/MyDrive/Chatgpt_KMLE/output/{file[:file.find('.')]}_score.xlsx", index=False)

    case_list.append(file[:file.find(".")])
    acc_list.append(round(df4["check"].sum()/360*100, 5))

score_df = pd.DataFrame({"Case":case_list, "Accuracy":acc_list})
score_df.sort_values("Accuracy", ascending=False).reset_index(drop=True)